In [5]:
import numpy as np
import torch
from torch import nn, Tensor
import autograd.numpy as np1
from autograd import grad

In [3]:
def plot_solution_space(target_space, target_space_full, target_space_cir):
    fig, axes = plt.subplots(2, 1, figsize=(6, 6), sharex=True, sharey=True)

    axes[0].scatter(target_space_full[:, 0], target_space_full[:, 1], s=10, c="red", label="X - Constraints")
    axes[0].legend()

    axes[0].scatter(target_space_cir[:, 0], target_space_cir[:, 1], s=10, c="yellow", label="Circle - (f - constraint)")
    axes[0].legend()

    axes[1].scatter(target_space[:, 0], target_space[:, 1], s=10, c="gray", label="Feasible solutions space")
    axes[1].legend()

    # Đảm bảo tỷ lệ trục bằng nhau
    for ax in axes:
        ax.set_aspect('equal', adjustable='box')

    plt.tight_layout()
    plt.show()

In [4]:
class Problem():
    def __init__(self, resolution=1000):
        self.target_space, self.target_space_full, self.target_space_cir, self.sols = self.create_pf(resolution)
        plot_solution_space(self.target_space, self.target_space_full, self.target_space_cir)
    # --- Objective Functions ---
    @staticmethod
    def f1(x):
        return 4 * x[0][0]**2 + 4 * x[0][1]**2

    @staticmethod
    def f2(x):
        return (x[0][0] - 5)**2 + (x[0][1] - 5)**2

    def f(self, x):
        return max(self.f1(x), self.f2(x))
    
    # --- Constraint Functions ---
    @staticmethod
    def g1(x):
        return (x[0][0] - 5)**2 + x[0][1]**2 - 25

    @staticmethod
    def g2(x):
        return (x[0][0] - 8)**2 + (x[0][1] + 3)**2 - 7.7

    @staticmethod
    def g3(x):
        return -15 - x[0][0]

    @staticmethod
    def g4(x):
        return x[0][0] - 30

    @staticmethod
    def g5(x):
        return -15 - x[0][1]

    @staticmethod
    def g6(x):
        return x[0][1] - 30

    # Ràng buộc trên không gian mục tiêu  
    @staticmethod
    def g7(x):
        return (Problem.f1(x) - 50)**2 + (Problem.f2(x) - 50)**2 - 50**2

In [ ]:
from neural_ode import ODEF
def sign(g):
    if g>0:
        return torch.tensor([1])
    elif g<0:
        return torch.tensor([0])
    else:
        p = torch.rand(1)
        return p
    
class TestODEF(ODEF):
    def __init__(self, A, B, x0,s,r):
        super(TestODEF, self).__init__()
        self.A = nn.Linear(2, 2, bias=False)
        self.A.weight = nn.Parameter(A)
        self.B = nn.Linear(2, 2, bias=False)
        self.B.weight = nn.Parameter(B)
        self.x0 = nn.Parameter(x0)
        self.s = s
        self.r = r
    def forward(self, x, t):
        # print(x)
        g1, g2, g3, g4, g5, g6 = Problem.g1, Problem.g2, Problem.g3, Problem.g4, Problem.g5, Problem.g6
        g7 = Problem.g7
        J = torch.tensor(
            [
                sign(g1(x)).item(),
                sign(g2(x)).item(),
                sign(g3(x)).item(),
                sign(g4(x)).item(),
                sign(g5(x)).item(),
                sign(g6(x)).item(),
                sign(g7(x)).item(),
            ]
        )
        c = torch.prod(1-J)
        z = x.detach().cpu().numpy()
        grad_f = grad(self.s)(z,self.r)
        grad_g1 = grad(g1)(z)
        grad_g2 = grad(g2)(z)
        grad_g3 = grad(g3)(z)
        grad_g4 = grad(g4)(z)
        grad_g5 = grad(g5)(z)
        grad_g6 = grad(g6)(z)
        grad_g7 = grad(g7)(z)

        dxdt = -c*torch.tensor(grad_f) - (
            sign(g1(x))*torch.tensor(grad_g1)
            + sign(g2(x))*torch.tensor(grad_g2)
            + sign(g3(x))*torch.tensor(grad_g3)
            + sign(g4(x))*torch.tensor(grad_g4)
            + sign(g5(x))*torch.tensor(grad_g5)                                        
            + sign(g6(x))*torch.tensor(grad_g6)
            + sign(g7(x))*torch.tensor(grad_g7)
        ) 
        return dxdt